In [20]:
#%%
import os
import numpy as np
import datetime
from matplotlib import pyplot as plt

# import the Obspy modules that we will use in this exercise
import obspy
from obspy.clients.fdsn import Client
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees

In [27]:
#%% function to download the seismograms give the network, station, startime and endtime of the waveform
def download_stations(network, stations, channels, path, location='10'):

    
    for station in stations:
        for channel in channels:
            fname = path + "/" + network + "." + station + "." + channel 
            inventory = client.get_stations(network=network, station=station, channel=channel, 
                                  location=location,level="response")
            inventory.write(fname + ".xml", format="STATIONXML") 

# download waveform given network          
def download_waveforms(network, stations, channels, starttime, endtime, path, location='10'):
    
    for station in stations:
        for channel in channels:
            fname = path + "/" + network + "." + station + "." + channel + "." + starttime.strftime('%Y%m%d')
            tr = client.get_waveforms(network=network, station=station, channel=channel, 
                                      location=location, starttime = starttime - 1800, endtime=endtime + 1800,
                                      attach_response=True)
            tr.detrend("spline", order=3, dspline=1000)
            tr.remove_response(output="VEL")
            
            # here to deal with the taperring at both end, only keep the central 1-hour long data
            newtr = tr.slice(starttime, endtime) # probably fix later
            
            newtr.write(fname + ".mseed", format='MSEED')
            

In [22]:
#%%
working_dir = '/Users/Yin9xun/Work/island_stations'
if not os.path.exists(working_dir):
    os.makedirs(working_dir)
os.chdir(working_dir)

print(os.getcwd())

/Users/Yin9xun/Work/island_stations


In [23]:
#%%
#@title Step 2.1 Specify the stations and time { display-mode: "both" }
# Specify the network and stations we want.
network = 'IU' # A specific seismic network to which the stations belong 
stations = np.array(['XMAS']) # Names of the stations
channels = np.array(['BHE','BHN','BHZ']) # Channels
# Specify begining time of the waveforms
year = 2011 
month = 3 
day = 11 
hour = 0 
minute = 0 
second = 0 
seconds_per_day = 24 * 60 * 60



station_dir = working_dir + '/stations'
if not os.path.exists(station_dir):
    os.makedirs(station_dir)
waveform_dir = working_dir + '/waveforms'
if not os.path.exists(waveform_dir):
    os.makedirs(waveform_dir)


In [24]:
#%%
client = Client('IRIS', debug=True)

Installed new opener with handlers: [<obspy.clients.fdsn.client.CustomRedirectHandler object at 0x7fc37b21dd68>]
Base URL: http://service.iris.edu
Request Headers: {'User-Agent': 'ObsPy/1.2.2 (Darwin-20.3.0-x86_64-i386-64bit, Python 3.6.13)'}
Loading discovered services from cache.


In [25]:
download_stations(network, stations, channels, station_dir)

Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHE&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHE&level=response with HTTP code: 200
Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHN&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHN&level=response with HTTP code: 200
Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHZ&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?network=IU&station=XMAS&location=10&channel=BHZ&level=response with HTTP code: 200


In [28]:
# Set the time of waveform we want (1 hour long)
starttime=obspy.UTCDateTime(year, month, day, hour, minute, second)
endtime=obspy.UTCDateTime(year, month, day+1, hour, minute, second)
download_waveforms(network, stations, channels, starttime, endtime, waveform_dir)

Downloaded http://service.iris.edu/fdsnws/dataselect/1/query?starttime=2011-03-10T23%3A30%3A00.000000&endtime=2011-03-12T00%3A30%3A00.000000&network=IU&station=XMAS&location=10&channel=BHE with HTTP code: 200
Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998152&network=IU&station=XMAS&location=10&channel=BHE&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998152&network=IU&station=XMAS&location=10&channel=BHE&level=response with HTTP code: 200


/Users/Yin9xun/opt/anaconda3/envs/spyder-py36/lib/python3.6/site-packages/obspy/io/mseed/core.py:790: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


Downloaded http://service.iris.edu/fdsnws/dataselect/1/query?starttime=2011-03-10T23%3A30%3A00.000000&endtime=2011-03-12T00%3A30%3A00.000000&network=IU&station=XMAS&location=10&channel=BHN with HTTP code: 200
Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998146&network=IU&station=XMAS&location=10&channel=BHN&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998146&network=IU&station=XMAS&location=10&channel=BHN&level=response with HTTP code: 200


/Users/Yin9xun/opt/anaconda3/envs/spyder-py36/lib/python3.6/site-packages/obspy/io/mseed/core.py:790: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


Downloaded http://service.iris.edu/fdsnws/dataselect/1/query?starttime=2011-03-10T23%3A30%3A00.000000&endtime=2011-03-12T00%3A30%3A00.000000&network=IU&station=XMAS&location=10&channel=BHZ with HTTP code: 200
Uncompressing gzipped response for http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998146&network=IU&station=XMAS&location=10&channel=BHZ&level=response
Downloaded http://service.iris.edu/fdsnws/station/1/query?starttime=2011-03-10T23%3A30%3A00.023152&endtime=2011-03-12T00%3A29%3A59.998146&network=IU&station=XMAS&location=10&channel=BHZ&level=response with HTTP code: 200


/Users/Yin9xun/opt/anaconda3/envs/spyder-py36/lib/python3.6/site-packages/obspy/io/mseed/core.py:790: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
